In [0]:
!python -m spacy download en_core_web_lg
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 826.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=fc654b50d7baf57336210db98a5c23d60c7b1e33be46643f4eb94df76c957fe0
  Stored in directory: /tmp/pip-ephem-wheel-cache-63b6qy5o/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mou

In [0]:
#import py_compile


#script = "/content/drive/My Drive/kodlar/kod.py"
#py_compile.compile(script)

import spacy
nlp = spacy.load('en_core_web_lg')

from spacy.matcher import Matcher

from collections import Counter



words = input("Insert claim 1 or description text here:  ")



doc = nlp(words)

#for ent in doc.ents:
#  print("ents: ", ent.text, " ", ent.label_)

#nlp.tokenizer.add_special_case('-', [{ORTH: '-', POS:ADP}])

matcher = Matcher(nlp.vocab)
#pattern 00,01,10,11 : referanslar için (ör: a chassis, the chassis, vehicle chassis)
pattern00 = [{'POS': 'DET', 'OP': '+'},{'POS':'NOUN'},{'LIKE_NUM': True}]
pattern01 = [{'POS':'NOUN'},{'POS':'NOUN'},{'LIKE_NUM': True}]
pattern02 = [{'POS':'ADJ'},{'POS':'NOUN'},{'LIKE_NUM': True}]
pattern10 = [{'POS': 'DET', 'OP': '+'},{'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True},{'TEXT':')'}]
pattern11 = [{'POS':'NOUN'},{'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True},{'TEXT':')'}]
pattern12 = [{'POS':'ADJ'},{'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True},{'TEXT':')'}]

#pattern 2,3,4 : noun ve noun_phrase için sonrasında frekans belirlenerek SEARCH query keyword belirlemede kullanılıyor

pattern2 = [{'LIKE_NUM': True,'IS_DIGIT': False}, {'POS':'NOUN'}]
pattern3 = [{'POS':'NOUN'},{'POS':'NOUN'}]
#pattern4 = [{'POS':'ADJ'},{'POS':'NOUN'}]

matcher.add('REFERENCE',None,pattern00,pattern01,pattern02,pattern10,pattern11,pattern12)
found_matches = matcher(doc)

print("\nPOSSIBLE REFERENCES : \n")
for match_id, start, end in found_matches:
  string_id = nlp.vocab.strings[match_id]  # get string representation
  span = doc[start:end]                    # get the matched span
  print(match_id, string_id, start, end, span.text)

print('\n')


if len(found_matches)==0:
  print("NO POSSIBLE REFERENCE found with given format\n")

referenceList = [doc[start:end].text for match_id, start, end in found_matches]
referenceList_freq = Counter(referenceList)
common_words = referenceList_freq.most_common(40)
print("\nPOSSIBLE REFERENCES (reference_phrase,freq): ", common_words,"\n")

print("SORTED POSSIBLE REFERENCES REGARDING FREQUENCY IN DESCENDING ORDER: \n")

sorted_reference_list = [word for (word, freq) in common_words]
for i in range(0,len(sorted_reference_list)):
  print(str(i+1)+")-  "+sorted_reference_list[i])

print("\n")

matcher.remove('REFERENCE')
matcher.add('NOUN_PHRASE',None,pattern2,pattern3)
found_matches = matcher(doc)
print("POSSIBLE NOUN PHRASES:\n")
for match_id, start, end in found_matches:
  string_id = nlp.vocab.strings[match_id]  # get string representation
  span = doc[start:end]                    # get the matched span
  print(match_id, string_id, start, end, span.text)
    
    
startDocText = [doc[start:end].text for match_id, start, end in found_matches]


unexpected_element=[]
try:
  for i in range(0,len(startDocText)):
    if "-" in startDocText[i]:
      #print("unexpected element = ")
      unexpected_element.append(startDocText[i])
except IndexError:
  print("ERROR:index out of range")

print("List for unexpected element:",unexpected_element)

startDocText = [x for x in startDocText if x not in unexpected_element]


print("\nCRITICAL IPC KEYWORDS (comprising system,arrangement,product,device,method etc.):\n")

try:
    #if (startDocText[i][0] != '-' or startDocText[i][-1] != '-'):
  for i in range(len(startDocText)):
    if (startDocText[i].split()[1].lower()== "system" or
        startDocText[i].split()[1].lower()=="arrangement" or
        startDocText[i].split()[1].lower()=="arangement" or startDocText[i].split()[1].lower()=="apparatus" or
        startDocText[i].split()[1].lower()=="aparatus" or startDocText[i].split()[1].lower()=="device" or
        startDocText[i].split()[1].lower()=="devices" or
        startDocText[i].split()[1].lower()=="method" or startDocText[i].split()[1].lower()=="product" or 
        startDocText[i].split()[1].lower()=="production" or startDocText[i].split()[1].lower()=="assembly" or
        startDocText[i].split()[1].lower()=="asembly") :
      print(startDocText[i])
      #critical_firstword=startDocText[i]

except: 
  print("AN EXCEPTION OCCURED. UNEXPECTED CHARACTER-INDEX ERROR\n")

  
startDocText_freq = Counter(startDocText)
common_words = startDocText_freq.most_common(40)
print("\nIPC KEYWORDS (noun_phrase,freq): ", common_words,"\n")

#*************** common_words liste lengthi 2'den küçükse "list index out of range" hatası veriyor. Dolayısıyla for ile len(list) şeklinde sınırlandırmak doğru olabilir
print("SORTED IPC KEYWORDS QUERY FORMAT REGARDING FREQUENCY IN DESCENDING ORDER:\n")

query_words=[]
try:
  for i in range(0,len(common_words)):
    #print(startDocText[i][0],startDocText[i][-1])
    #if (startDocText[i][0] != '-' or startDocText[i][-1] != '-'):
    phrase_firstword = common_words[i][0].split()[0].lower()
    phrase_secondword = common_words[i][0].split()[1].lower()
    if (phrase_secondword[-1]=="s"):
      phrase_secondword = common_words[i][0].split()[1][:-1].lower()
    query_word = phrase_firstword+"_"+phrase_secondword+"+"
    print(str(i+1)+")-  "+query_word)
    query_words.append(query_word)
except:
  print('ERROR - UNEXPECTED CHARACTER-INDEX ERROR\n')


unique_keyword_list = [word for (word, freq) in common_words if freq is 1]
print("\nUNIQUE KEYWORDS (freq=1): \n")
if (len(unique_keyword_list) ==0):
  print("NO UNIQUE KEYWORDS FOUND")
else:
  for i in range(len(unique_keyword_list)):
    print(unique_keyword_list[i])


#print("\nCRITICAL IPC KEYWORDS (comprising system,arrangement,product,device,method etc.):\n")
#for i in range(0,len(critical_keyword_list)):
 # if (critical_keyword_list[i].split()[1].lower()== "system" or
  #    critical_keyword_list[i].split()[1].lower()=="arrangement" or
   #   critical_keyword_list[i].split()[1].lower()=="arangement" or critical_keyword_list[i].split()[1].lower()=="apparatus" or
    #  critical_keyword_list[i].split()[1].lower()=="aparatus" or critical_keyword_list[i].split()[1].lower()=="device" or
     # critical_keyword_list[i].split()[1].lower()=="method" or critical_keyword_list[i].split()[1].lower()=="product" or 
      #critical_keyword_list[i].split()[1].lower()=="production" or critical_keyword_list[i].split()[1].lower()=="assembly" or
      #critical_keyword_list[i].split()[1].lower()=="asembly"):
    #print(critical_keyword_list[i])
 
  
print("\nPROBABLE PATENW QUERY FOR IPC OR SIMILAR DOCUMENTS : \n")
patenw_query = " "
if (len(query_words) > 3):
  for i in range(0,3):
    patenw_query += query_words[i]+","
  print("and"+patenw_query[:-1])

elif (len(query_words) > 0):
  for i in range(0,len(query_words)):
    patenw_query += query_words[i]+","
  print("and"+patenw_query[:-1])

else: 
  print("""NO NOUN PHRASE FOUND... NO RESULTS FOR QUERY !""")








In [0]:
from scipy import spatial

cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

wordforSimilarity= input("Insert a word for similar matches: ")
wordforSimilarity = wordforSimilarity.lower()
new_vector = nlp.vocab[wordforSimilarity].vector

computed_similarities = []

for word in nlp.vocab:
    # Ignore words without vectors and mixed-case words:
    if word.vector_norm > 0:
        if word.is_lower:
          if word.is_alpha:
            similarity = cosine_similarity(new_vector, word.vector)
            computed_similarities.append((word, similarity))

computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])

print([w[0].text for w in computed_similarities[:20]])


Insert a word for similar matches: WORD
['word', 'phrase', 'words', 'meaning', 'phrases', 'language', 'spoken', 'dictionary', 'spelling', 'referring', 'means', 'saying', 'verb', 'term', 'speak', 'sentence', 'spelled', 'meant', 'name', 'speaking']


In [0]:
#IPC-PREDICT-NOUN PHRASE

import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial

nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input=my_input.lower()


input_doc = nlp(my_input)

#for ent in doc.ents:
#  print("ents: ", ent.text, " ", ent.label_)

#nlp.tokenizer.add_special_case('-', [{ORTH: '-', POS:ADP}])

matcher = Matcher(nlp.vocab)
#pattern 00,01,10,11 : referanslar için (ör: a chassis, the chassis, vehicle chassis)
patternipc1 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc2 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc3 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'ADJ'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc4 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'ADJ'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc5 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
matcher.add('IPC',None,patternipc1,patternipc2,patternipc3,patternipc4,patternipc5)
found_matches = matcher(input_doc)

print('\nPOSSIBLEIPC WORDS:\n')
if len(found_matches)==0:
  print("NO POSSIBLE IPC WORD found with given format\n")

list_ipcwords = []
for match_id, start, end in found_matches:
  string_id = nlp.vocab.strings[match_id]  # get string representation
  span = input_doc[start:end]                    # get the matched span
  print(match_id, string_id, start, end, span.text)
  list_ipcwords.append(span.text)
print('\n')

ipctext = ' '.join(list_ipcwords)
rawdoc = nlp(ipctext)
apptext=[]
for token in rawdoc:
  if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
    apptext.append(token.text)
# unique ifadeler bulunup listeye dahil edilir, listeden tek bir string(unique_text) üretilir,bu string de nlp ile doc objesine dönüştürülür 
unique_list = [] 
for x in apptext:
  if x not in unique_list: 
    unique_list.append(x) 

unique_text = ' '.join(unique_list)
appdoc = nlp(unique_text)
print("\nAPPDOC: ",appdoc)


df = pd.read_csv('/content/drive/My Drive/ipc.tsv', sep='\t')

# ilk cümle başarısız olursa alttaki 2 yorum koduna devam edilebilir
#app = input("insert critical words for ipc: ")
#appdoc= nlp(app)
probIpc=[]
for i in(range(len(df['Definition']))):
  print(appdoc.similarity(nlp(str(df['Definition'][i]))))
  probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))

print("FIRST FIVE MAXIMUM VALUES:\n")

df['predictionValue']=pd.Series(probIpc)
print(df.sort_values(by='predictionValue', ascending = False))
print("\n")

try:
  print("VALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
except:
  print('NO PREDICTION VALUE GREATER THAN 0.5\n')


answer=input("\nIf you are not satisfied with the results do you want to insert your own noun phrases for ipc ?\n")

if answer.lower() == "y" or answer.lower() == "yes":
  app = input("Insert critical noun phrases for ipc wihout punctuation (e.g image processor, marine, communication network ): ")
  appdoc= nlp(app)
  probIpc=[]
  for i in(range(len(df['Definition']))):
    print(appdoc.similarity(nlp(str(df['Definition'][i]))))
    probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))
  print("\nMAXIMUM VALUES:\n")
  df['predictionValue']=pd.Series(probIpc)
  print(df.sort_values(by='predictionValue', ascending = False).head(20))
  print("\n")
  
  try:
    print("VALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
    print("\nYou can run again for new search...")
  except:
    print('NO PREDICTION VALUE GREATER THAN 0.5\n')

else:
  print("\nYou can run again for new search...")





In [0]:
df.sort_values(by='predictionValue', ascending = False).head(40)



In [0]:
df.tail(60)

In [0]:
app=input("app: ")
ipc=input("ipc: ")
docipc = nlp(ipc)
docapp = nlp(app)
print(docapp.vector_norm,docipc.vector_norm)
print("app:",docapp.has_vector)
print("ipc:",docipc.has_vector)
print(docapp.similarity(docipc))

In [0]:
doc=nlp("an airbag deopleyment system is provided")
for chunk in doc.noun_chunks:
  liste=[chunk.text for chunk in doc.noun_chunks]
print("liste: ",liste)
for token in doc:
  print(token.text,token.lemma_,token.pos_,token.tag_,token.dep_,token.shape_,token.is_alpha,token.is_stop)

liste:  ['an airbag deopleyment system']
an an DET DT det xx True True
airbag airbag NOUN NN compound xxxx True False
deopleyment deopleyment NOUN NN compound xxxx True False
system system NOUN NN nsubjpass xxxx True False
is be VERB VBZ auxpass xx True True
provided provide VERB VBN ROOT xxxx True False


In [0]:
#IPC PREDICT-NOUN-CHUNKS (+ (TOKEN.TAG_=VBG) ?)

import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial

nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input=my_input.lower()

#--------
input_doc = nlp(my_input)
liste_chunk=[]
for chunk in input_doc.noun_chunks:
  print("CHUNK: ",chunk)
  for i in(range(len(chunk))):
    if (chunk[i].is_stop == False and chunk[i].text != "-" and chunk[i].is_punct == False and chunk[i].pos_ != 'NUM'):
      liste_chunk.append(chunk[i].text)
#---------

print("CHUNK LİSTESİ: ",liste_chunk)

ipctext = ' '.join(liste_chunk)


appdoc = nlp(ipctext)
print("\nAPPDOC: ",appdoc)


df = pd.read_csv('/content/drive/My Drive/ipc.tsv', sep='\t')


probIpc=[]
for i in(range(len(df['Definition']))):
  print(appdoc.similarity(nlp(str(df['Definition'][i]))))
  probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))

print("\nMAXIMUM VALUES: \n")

df['predictionValue']=pd.Series(probIpc)
print(df.sort_values(["predictionValue"], ascending = False).head(20))
print("\n")

#burası çok önemli yeni df'nin indexi sıfırlanmalı
df2 = df.sort_values(["predictionValue"], ascending = False)
df2 = df2.reset_index(drop=True)

#most prop ipc list, df2'deki IPC verileri ile oluşturulur
most_prob_ipc_list=[]
for i in range(0,20):
  most_prob_ipc_list.append((df2['IPC'][i],i))

final_ipc_df1 = pd.DataFrame(most_prob_ipc_list, columns =['IPC','FIRST-INDEX'])
#print("MOST PROBABLE IPC LIST: \n",most_prob_ipc_list)



try:
  print("\nVALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
except:
  print('NO PREDICTION VALUE GREATER THAN 0.5\n')


answer=input("\nIf you are not happy with the results do you want to insert your own noun phrases for ipc ?\n")

if answer.lower() == "y" or answer.lower() == "yes":
  app = input("Insert critical noun phrases for ipc wihout punctuation (e.g image processor, marine, communication network ): ")
  appdoc= nlp(app)
  probIpc=[]
  for i in(range(len(df['Definition']))):
    print(appdoc.similarity(nlp(str(df['Definition'][i]))))
    probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))
  print("\nMAXIMUM VALUES:\n")
  df['predictionValue']=pd.Series(probIpc)
  print(df.sort_values(["predictionValue"], ascending = False).head(20))

  print("\nFIRST PREDICTION VALUES:\n", df2.head(20))
  
  # ilk df2'dn farklı olarak ikinci df2 oluşturulur sonuçta her iki df2'deki IPC verileri ile most prob ipc list oluşur
  df2=df.sort_values(["predictionValue"], ascending = False)
  df2 = df2.reset_index(drop=True)

  most_prob_ipc_list2=[]
  for m in range(0,20):
    most_prob_ipc_list2.append((df2['IPC'][m],m))
  #print("MOST PROBABLE IPC LIST: \n",most_prob_ipc_list)
  final_ipc_df2 = pd.DataFrame(most_prob_ipc_list2, columns =['IPC','SECOND-INDEX'])
  result_df = pd.merge(final_ipc_df1,final_ipc_df2, on='IPC', how='outer', indicator=True)

  #result_df = pd.merge(final_ipc_df1,final_ipc_df2, on='IPC')
  #result_df = final_ipc_df1.append(final_ipc_df2, sort=False)
  print("\nRESULT:\n",result_df)
  #final_ipc_list = [(x, most_prob_ipc_list.count(x),most_prob_ipc_list.) for x in set(most_prob_ipc_list)]
  #print ("IPC FREQUENCY IN BOTH METHODS: ", final_ipc_list) 
  
  #final_ipc_df = pd.DataFrame(final_ipc_list, columns =['IPC', 'FREQUENCY', 'FIRST-INDEX']) 
  #-----alttaki komut önemli, son sorting
  result_df['TOTAL-INDEX'] = result_df['FIRST-INDEX']+result_df['SECOND-INDEX']

  final_sorted_ipc_df = result_df.sort_values(["TOTAL-INDEX"], ascending = True)
  print("\nIPC FREQUENCY IN BOTH METHODS:\n ",final_sorted_ipc_df)

  try:
    print("\nVALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
    print("\nYou can run again for new search...")
  except:
    print('NO PREDICTION VALUE GREATER THAN 0.5\n')

else:
  print("\nYou can run again for new search...")





In [2]:
#IPC PREDICT-NOUN-CHUNKS (+ (TOKEN.TAG_=VBG) ?)

import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial

nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input=my_input.lower()

#--------
input_doc = nlp(my_input)
liste_chunk=[]
for chunk in input_doc.noun_chunks:
  print("CHUNK: ",chunk)
  for i in(range(len(chunk))):
    if (chunk[i].is_stop == False and chunk[i].text != "-" and chunk[i].is_punct == False and chunk[i].pos_ != 'NUM'):
      liste_chunk.append(chunk[i].text)
#---------

print("CHUNK LİSTESİ: ",liste_chunk)

ipctext = ' '.join(liste_chunk)


appdoc = nlp(ipctext)
print("\nAPPDOC: ",appdoc)


df = pd.read_csv('/content/drive/My Drive/FULLIPC-22-11.tsv', sep='\t')

#BURADA DEF1 SÜTUNU İLE BENZERLİK DEĞERLERİ PROBIPC1 LİSTESİNE EKLENİR. DEF1 4 HANELİ IPC SINIFI İÇİN TANIMDIR
probIpc1=[]
for i in(range(len(df['IPC']))):
  #print(appdoc.similarity(nlp(str(df['DEF1'][i]))))
  probIpc1.append(appdoc.similarity(nlp(str(df['DEF1'][i]))))

print("\nMAXIMUM VALUES FOR DEF1: \n")

df['predictionValue1']=pd.Series(probIpc1)
print(df.sort_values(["predictionValue1"], ascending = False).head(20))
print("\n")

#BURADA DEF2 SÜTUNU İLE BENZERLİK DEĞERLERİ PROBIPC2 LİSTESİNE EKLENİR. DEF2 3 HANELİ IPC SINIFI İÇİN TANIMDIR
probIpc2=[]
for i in(range(len(df['IPC']))):
  #print(appdoc.similarity(nlp(str(df['DEF2'][i]))))
  probIpc2.append(appdoc.similarity(nlp(str(df['DEF2'][i]))))

print("\nMAXIMUM VALUES FOR DEF2: \n")

df['predictionValue2']=pd.Series(probIpc2)
print(df.sort_values(["predictionValue2"], ascending = False).head(20))
print("\n")

#burası çok önemli yeni df'nin indexi sıfırlanmalı
df['FULLIPC-Prediction']=df['predictionValue1']+df['predictionValue2']
df2 = df.sort_values(["FULLIPC-Prediction"], ascending = False)
df2 = df2.reset_index(drop=True)
#---------------------
print("FULL IPC PREDICTION MAX. VALUES:\n",df2)






Insert the first meaningful sentence in description regarding ipc: The present invention is directed to an improved high beam headlight indicator and, more specifically, to a high beam headlight indicator mounted dead center on the top of the steering wheel facing the driver of a vehicle
CHUNK:  the present invention
CHUNK:  an improved high beam headlight indicator
CHUNK:  a high beam headlight indicator
CHUNK:  dead center
CHUNK:  the top
CHUNK:  the steering wheel
CHUNK:  the driver
CHUNK:  a vehicle
CHUNK LİSTESİ:  ['present', 'invention', 'improved', 'high', 'beam', 'headlight', 'indicator', 'high', 'beam', 'headlight', 'indicator', 'dead', 'center', 'steering', 'wheel', 'driver', 'vehicle']

APPDOC:  present invention improved high beam headlight indicator high beam headlight indicator dead center steering wheel driver vehicle

MAXIMUM VALUES FOR DEF1: 

     IPC  ... predictionValue1
12  B60Q  ...         0.709329
15  B60T  ...         0.707133
13  B60R  ...         0.705890
2  